In [1]:
import io
with io.open("rebuild_from_archive_Marlin_I3.cpp.hex.dis") as f:
    asm_lines = f.read().splitlines()

In [2]:
print(asm_lines[0:10])

['', 'rebuild_from_archive_Marlin_I3.cpp.hex:     file format ihex', '', '', 'Disassembly of section .sec1:', '', '00000000 <.sec1>:', '       0:\t0c 94 07 0e \tjmp\t0x1c0e\t;  0x1c0e', '       4:\t0c 94 34 0e \tjmp\t0x1c68\t;  0x1c68', '       8:\t0c 94 34 0e \tjmp\t0x1c68\t;  0x1c68']


In [3]:
import re
jumps=re.compile('(r|e?i)?(jmp)')
calls=re.compile('(r|e?i)?(call)')
branches=re.compile('br([lg][te]|eq|ne|lo|sh|[bctv][cs]|mi|pl|i[ed])')
skips=re.compile('cpse|sb[rib][cs]')
returns=re.compile('reti?')

In [84]:
targets = set()
links = set()
for (addr,bytecode,nmem,arg,*comment) in [ l.split('\t') for l in asm_lines if len(l.split('\t'))>=4]:
    #print('Asm at {} is a {} {}'.format(addr,nmem,arg))
    naddr=int(addr.strip().translate({ord(c): None for c in ':!@#$'}),base=16)& 0xffff
    if (jumps.match(nmem)):
        narg=int(arg.strip().translate({ord(c): None for c in '."'}),base=16)& 0xffff
        if ('+' in arg or '-' in arg):
            narg=naddr+narg
        targets.add(naddr)
        targets.add(narg)
        links.add((naddr,narg))
        print(f"BOING {naddr:x} -> {narg:x}")
    if (calls.match(nmem)):
        narg=int(arg.strip().translate({ord(c): None for c in '."'}),base=16)& 0xffff
        if ('+' in arg or '-' in arg):
            narg=naddr+narg
        targets.add(naddr)
        targets.add(narg)
        links.add((naddr,narg))
        links.add((narg,naddr))
        print(f"BINGBOING {naddr:x} <-> {narg:x}")
    if (branches.match(nmem)):
        dest=int(arg.strip().translate({ord(c): None for c in '."'}),base=16)& 0xffff
        if ('+' in arg or '-' in arg):
            dest=naddr+dest+2
        targets.add(naddr)
        targets.add(dest)
        links.add((naddr,dest))
        print(f"BING {naddr:x} -> {dest:x}")
        targets.add((naddr+2))
        links.add((naddr,naddr+2))
        print(f"BING {naddr:x} -> {naddr+2:x}")
        
        
#print("All targets:")
#print([f'{target:x}' for target in targets])
#print("All links:")
#print([f'({a:x},{b:x})' for (a,b) in links])

BOING 0 -> 1c0e
BOING 4 -> 1c68
BOING 8 -> 1c68
BOING c -> 1c68
BOING 10 -> 1c68
BOING 14 -> 1c68
BOING 18 -> 1c68
BOING 1c -> 1c68
BOING 20 -> 1c68
BOING 24 -> 1c68
BOING 28 -> 1c68
BOING 2c -> 1c68
BOING 30 -> 1c68
BOING 34 -> 9fac
BOING 38 -> 1c68
BOING 3c -> 1c68
BOING 40 -> 1c68
BOING 44 -> f118
BOING 48 -> 1c68
BOING 4c -> 1c68
BOING 50 -> 1c68
BOING 54 -> 1c68
BOING 58 -> 41c
BOING 5c -> 982a
BOING 60 -> 1c68
BOING 64 -> 4c74
BOING 68 -> 1c68
BOING 6c -> 1c68
BOING 70 -> 1c68
BOING 74 -> 1c68
BOING 78 -> 1c68
BOING 7c -> 1c68
BOING 80 -> ee34
BOING 84 -> 1c68
BOING 88 -> 1c68
BOING 8c -> 1c68
BOING 90 -> 1c68
BOING 94 -> 1c68
BOING 98 -> 1c68
BOING 9c -> 9138
BOING a0 -> 1c68
BOING a4 -> 1c68
BOING a8 -> edda
BOING ac -> 1c68
BOING b0 -> 1c68
BOING b4 -> 1c68
BOING b8 -> 1c68
BOING bc -> ed80
BOING c0 -> 1c68
BOING c4 -> 1c68
BOING c8 -> 1c68
BOING cc -> 1c68
BOING d0 -> 1c68
BOING d4 -> 1c68
BOING d8 -> 1c68
BOING dc -> 1c68
BOING e0 -> 1c68
BOING e8 -> f7e2
BINGBOING fa <-> d7

BING 255a -> 255e
BING 255a -> 255c
BOING 255c -> 2874
BOING 255e -> 2566
BOING 2564 -> 2584
BOING 2570 -> 2578
BOING 257c -> 2592
BOING 258c -> 25a2
BINGBOING 25a4 <-> 2938
BOING 25b2 -> 25d4
BING 25ba -> 25d0
BING 25ba -> 25bc
BOING 25be -> 25d2
BOING 25c2 -> 25d2
BOING 25c8 -> 25cc
BOING 25cc -> 25ce
BOING 25d2 -> 25f8
BING 25de -> 25e6
BING 25de -> 25e0
BOING 25e4 -> 25fa
BOING 25ea -> 25fa
BOING 25ec -> 25f2
BING 25f2 -> 25f6
BING 25f2 -> 25f4
BOING 25fa -> 2632
BOING 25fe -> 2626
BING 2602 -> 2608
BING 2602 -> 2604
BOING 2606 -> 2626
BOING 2610 -> 2620
BINGBOING 2618 <-> 27a8
BING 261e -> 1260c
BING 261e -> 2620
BOING 2620 -> 2628
BING 2624 -> 262a
BING 2624 -> 2626
BOING 2628 -> 262a
BOING 262e -> 2656
BINGBOING 2636 <-> 2796
BOING 263a -> 268e
BOING 263e -> 2644
BOING 2644 -> 266c
BOING 264a -> 266c
BING 2650 -> 2684
BING 2650 -> 2652
BOING 2654 -> 2658
BOING 2658 -> 265a
BINGBOING 2664 <-> 2778
BOING 2666 -> 2676
BINGBOING 266e <-> 2772
BING 2674 -> 12662
BING 2674 -> 2676
BIN

BOING 4f34 -> 4f64
BINGBOING 4f42 <-> 14ee0
BINGBOING 4f5c <-> 14b14
BINGBOING 4f84 <-> 14e5c
BINGBOING 4fa2 <-> 14e4a
BINGBOING 4fb2 <-> 14e3e
BINGBOING 4fd4 <-> 14d70
BINGBOING 4fd8 <-> 14f64
BINGBOING 5012 <-> a88a
BING 5018 -> 503e
BING 5018 -> 501a
BINGBOING 5028 <-> 14d96
BOING 503c -> 505e
BINGBOING 504a <-> a898
BING 5056 -> 15032
BING 5056 -> 5058
BINGBOING 505c <-> a6d8
BINGBOING 5064 <-> a972
BINGBOING 506c <-> a9ca
BINGBOING 5078 <-> a6d6
BINGBOING 508a <-> 14c44
BING 508e -> 5192
BING 508e -> 5090
BOING 5094 -> 50a8
BOING 509c -> 15094
BING 50a8 -> 1508a
BING 50a8 -> 50aa
BOING 50aa -> 511e
BINGBOING 50b8 <-> ae4a
BINGBOING 50c0 <-> a968
BINGBOING 50da <-> 14c6a
BINGBOING 50e0 <-> a9ce
BINGBOING 50ec <-> a6d6
BING 50f8 -> 15082
BING 50f8 -> 50fa
BOING 5112 -> 14e12
BINGBOING 5140 <-> a88a
BING 5146 -> 5150
BING 5146 -> 5148
BINGBOING 516c <-> a88a
BING 5172 -> 517c
BING 5172 -> 5174
BINGBOING 5198 <-> a88a
BING 519e -> 51a8
BING 519e -> 51a0
BINGBOING 51c4 <-> aca6
BING 51

BINGBOING 682a <-> 12e98
BING 682e -> 6834
BING 682e -> 6830
BOING 6830 -> 8a7a
BINGBOING 6834 <-> 13eec
BINGBOING 684e <-> ccd8
BOING 6852 -> 8a7a
BINGBOING 685a <-> 2b76
BOING 685e -> 8a7a
BINGBOING 6870 <-> 2268
BING 6878 -> 68ea
BING 6878 -> 687a
BINGBOING 689e <-> 2268
BINGBOING 68a6 <-> 2268
BINGBOING 68d4 <-> 2f98
BOING 68d8 -> 8a7a
BING 68e2 -> 68e8
BING 68e2 -> 68e4
BOING 68e4 -> 8a7a
BINGBOING 68ee <-> 2a5a
BINGBOING 6900 <-> 2268
BING 6908 -> 697a
BING 6908 -> 690a
BINGBOING 692e <-> 2268
BINGBOING 6936 <-> 2268
BINGBOING 6960 <-> 2c2a
BOING 6964 -> 8a7a
BING 696e -> 6974
BING 696e -> 6970
BINGBOING 6970 <-> f0d6
BINGBOING 698a <-> 2268
BINGBOING 6996 <-> 2268
BING 699c -> 69a6
BING 699c -> 699e
BING 69aa -> 69b4
BING 69aa -> 69ac
BINGBOING 69b6 <-> 5894
BING 69c8 -> 69cc
BING 69c8 -> 69ca
BING 69d2 -> 69d8
BING 69d2 -> 69d4
BOING 69d4 -> 8a7a
BINGBOING 69e4 <-> 2f98
BINGBOING 69ea <-> 5894
BING 69f0 -> 6a36
BING 69f0 -> 69f2
BING 69fe -> 6a30
BING 69fe -> 6a00
BINGBOING 6a0

BOING 82f0 -> 182e8
BING 82fc -> 182de
BING 82fc -> 82fe
BINGBOING 830e <-> 29e
BINGBOING 831e <-> 5112
BOING 8326 -> 833a
BOING 832e -> 18326
BING 833a -> 1831c
BING 833a -> 833c
BINGBOING 8354 <-> 5112
BOING 835c -> 8370
BOING 8364 -> 1835c
BING 8370 -> 18352
BING 8370 -> 8372
BOING 8378 -> 18370
BOING 8380 -> 9b04
BINGBOING 8384 <-> 5894
BING 838a -> 839c
BING 838a -> 838c
BOING 8394 -> 8398
BINGBOING 8396 <-> 5cc8
BINGBOING 839a <-> a33a
BOING 839e -> 9af2
BINGBOING 83a2 <-> 5894
BING 83a8 -> 83b0
BING 83a8 -> 83aa
BOING 83ae -> 83e4
BINGBOING 83b0 <-> 5cc8
BINGBOING 83b4 <-> a968
BOING 83bc -> 83de
BOING 83d2 -> 83f2
BINGBOING 83ea <-> 5894
BING 83f0 -> 83fa
BING 83f0 -> 83f2
BINGBOING 83f2 <-> 5cc8
BINGBOING 83fc <-> 5894
BING 8402 -> 840a
BING 8402 -> 8404
BOING 8408 -> 8418
BINGBOING 840a <-> 5cc8
BINGBOING 840e <-> a968
BINGBOING 841a <-> ea6
BOING 841e -> 9a44
BINGBOING 8420 <-> f0d6
BOING 8424 -> 9a44
BINGBOING 8426 <-> 42ce
BINGBOING 842a <-> 4624
BOING 842e -> 9a3e
BINGBOI

...

BING e402 -> e414
BING e402 -> e404
BOING e40c -> e430
BING e410 -> e414
BING e410 -> e412
BOING e412 -> e554
BING e424 -> 1e414
BING e424 -> e426
BING e44e -> e462
BING e44e -> e450
BINGBOING e456 <-> 1d216
BING e45a -> e4cc
BING e45a -> e45c
BING e460 -> e484
BING e460 -> e462
BOING e476 -> e49e
BOING e4a0 -> e4a2
BINGBOING e4ca <-> e4ca
BINGBOING e4fa <-> 1e1f6
BING e4fe -> e578
BING e4fe -> e500
BINGBOING e50e <-> 1df7c
BING e512 -> e56c
BING e512 -> e514
BING e522 -> e544
BING e522 -> e524
BING e534 -> 1e4ce
BING e534 -> e536
BOING e536 -> e54e
BING e548 -> 1e4c2
BING e548 -> e54a
BOING e54c -> e54e
BINGBOING e58e <-> e58e
BINGBOING e5c6 <-> 1e0be
BING e5ca -> e644
BING e5ca -> e5cc
BOING e5d4 -> e5dc
BING e5e0 -> 1e5d0
BING e5e0 -> e5e2
BING e5f6 -> e60a
BING e5f6 -> e5f8
BING e600 -> 1e590
BING e600 -> e602
BOING e602 -> e612
BING e60c -> 1e58a
BING e60c -> e60e
BING e6b4 -> e6ce
BING e6b4 -> e6b6
BOING e6c6 -> e6f8
BING e6e2 -> e6e8
BING e6e2 -> e6e4
BING e70a -> e70e
BING e70a

BOING 1546 -> 1153e
BOING 1560 -> 1574
BOING 156e -> 1582
BOING 1596 -> 1598
BINGBOING 15aa <-> a6d8
BINGBOING 15ba <-> aca6
BING 15c0 -> 15fa
BING 15c0 -> 15c2
BOING 15c8 -> 15de
BOING 15d0 -> 115c8
BING 15dc -> 115be
BING 15dc -> 15de
BOING 15e4 -> 115dc
BOING 15e6 -> 193e
BINGBOING 15e8 <-> 98ba
BING 1604 -> 1608
BING 1604 -> 1606
BOING 1606 -> 176e
BOING 160e -> 1642
BOING 161a -> 1630
BOING 1622 -> 1161a
BING 162e -> 11610
BING 162e -> 1630
BOING 1630 -> 1664
BOING 163e -> 1654
BOING 1646 -> 1163e
BING 1652 -> 11634
BING 1652 -> 1654
BINGBOING 1666 <-> 5112
BOING 1670 -> 1686
BOING 1678 -> 11670
BING 1684 -> 11666
BING 1684 -> 1686
BINGBOING 1692 <-> 4f6c
BOING 169c -> 11694
BINGBOING 16a4 <-> 98ba
BINGBOING 16b0 <-> 98ba
BINGBOING 16b8 <-> 98ba
BING 16f4 -> 172e
BING 16f4 -> 16f6
BOING 16fc -> 1712
BOING 1704 -> 116fc
BING 1710 -> 116f2
BING 1710 -> 1712
BOING 1718 -> 11710
BOING 171a -> 176a
BING 1728 -> 17bc
BING 1728 -> 172a
BOING 1730 -> 1746
BOING 1738 -> 11730
BING 1744 -> 

BING 329c -> 32ae
BING 329c -> 329e
BOING 32a6 -> 32ae
BINGBOING 32b2 <-> 1e34
BING 32b8 -> 32ec
BING 32b8 -> 32ba
BING 32d0 -> 32da
BING 32d0 -> 32d2
BINGBOING 32d2 <-> f26a
BOING 32d8 -> 430a
BINGBOING 32da <-> a32c
BING 32e0 -> 32f2
BING 32e0 -> 32e2
BING 32e8 -> 32ec
BING 32e8 -> 32ea
BOING 32ea -> 4480
BING 32f0 -> 32f4
BING 32f0 -> 32f2
BOING 32f2 -> 3544
BING 32fa -> 335e
BING 32fa -> 32fc
BINGBOING 3314 <-> b092
BING 3320 -> 3332
BING 3320 -> 3322
BOING 332a -> 3332
BINGBOING 3336 <-> 1e34
BING 333c -> 3340
BING 333c -> 333e
BOING 333e -> 3642
BINGBOING 3358 <-> b092
BING 3364 -> 3368
BING 3364 -> 3366
BOING 3366 -> 35ca
BINGBOING 3368 <-> 22f0
BOING 3370 -> 3bf0
BING 3378 -> 33dc
BING 3378 -> 337a
BINGBOING 3392 <-> b092
BING 339e -> 33b0
BING 339e -> 33a0
BOING 33a8 -> 33b0
BINGBOING 33b4 <-> 1e34
BING 33ba -> 33be
BING 33ba -> 33bc
BOING 33bc -> 3534
BINGBOING 33d6 <-> b092
BING 33e2 -> 33e6
BING 33e2 -> 33e4
BOING 33e4 -> 351c
BINGBOING 33e6 <-> 22f0
BOING 33ee -> 3b42
BING

BING 62ee -> 6370
BING 62ee -> 62f0
BINGBOING 6308 <-> b092
BING 6314 -> 6336
BING 6314 -> 6316
BINGBOING 631a <-> 198a
BOING 6328 -> 6340
BINGBOING 632e <-> 198a
BINGBOING 633c <-> 3ad0
BING 6342 -> 6346
BING 6342 -> 6344
BOING 6344 -> 6944
BINGBOING 635e <-> b092
BING 636a -> 636e
BING 636a -> 636c
BOING 636c -> 68cc
BINGBOING 636e <-> 22f0
BOING 63ec -> 6660
BING 63f0 -> 63f4
BING 63f0 -> 63f2
BOING 63f2 -> 6686
BING 63fa -> 647c
BING 63fa -> 63fc
BINGBOING 6414 <-> b092
BING 6420 -> 6442
BING 6420 -> 6422
BINGBOING 6426 <-> 198a
BOING 6434 -> 644c
BINGBOING 643a <-> 198a
BINGBOING 6448 <-> 3ad0
BING 644e -> 6452
BING 644e -> 6450
BOING 6450 -> 6782
BINGBOING 646a <-> b092
BING 6476 -> 647a
BING 6476 -> 6478
BOING 6478 -> 670a
BINGBOING 647a <-> 22f0
BOING 6518 -> 675c
BING 651c -> 6520
BING 651c -> 651e
BOING 651e -> 6644
BING 6526 -> 658a
BING 6526 -> 6528
BINGBOING 6540 <-> b092
BING 654c -> 655e
BING 654c -> 654e
BOING 6556 -> 655e
BINGBOING 6562 <-> 1e34
BING 6568 -> 65bc
BING 

BOING 9368 -> 94da
BOING 9382 -> 94c4
BING 938a -> 939e
BING 938a -> 938c
BOING 93a4 -> 1939c
BOING 93c2 -> 9452
BING 93e2 -> 93f4
BING 93e2 -> 93e4
BING 9410 -> 9416
BING 9410 -> 9412
BOING 9414 -> 9416
BOING 941c -> 944c
BOING 9424 -> 943c
BOING 9436 -> 1942e
BINGBOING 946e <-> b7be
BINGBOING 9474 <-> a594
BINGBOING 947c <-> b7b2
BINGBOING 9482 <-> a588
BINGBOING 948a <-> b7ac
BINGBOING 9490 <-> a522
BINGBOING 94c2 <-> adca
BOING 94c8 -> 94cc
BING 94d0 -> 194ca
BING 94d0 -> 94d2
BINGBOING 94d8 <-> b71c
BING 94e4 -> 194a4
BING 94e4 -> 94e6
BINGBOING 94ea <-> 193b2
BINGBOING 9528 <-> ad2e
BOING 952e -> 9532
BING 9536 -> 19530
BING 9536 -> 9538
BINGBOING 953e <-> b680
BING 954a -> 1950a
BING 954a -> 954c
BINGBOING 9550 <-> 19310
BINGBOING 9572 <-> b602
BING 9578 -> 957e
BING 9578 -> 957a
BINGBOING 957c <-> b5fc
BOING 9582 -> 958a
BINGBOING 9588 <-> 19346
BOING 958a -> 95ae
BING 9598 -> 19592
BING 9598 -> 959a
BINGBOING 959c <-> 1943c
BINGBOING 95a2 <-> 1943c
BINGBOING 95ae <-> 19538
BOI

In [15]:
%matplotlib inline

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
G=nx.Graph()
G.add_edges_from(links)
G.add_nodes_from(targets)

In [ ]:
G.size()

In [ ]:
#takes too long nx.draw_spring(G)

In [85]:
import json
jsondata = {"nodes": [ {"name": f'ln{x & 0xffff:x}', "group":1} for x in sorted(targets) ], "links": [ {"source":f'ln{x & 0xffff:x}',"target":f'ln{y & 0xffff:x}', "weight":5} for (x,y) in links ] }


In [86]:
print(f'{[ f"{t& 0xffff:x}" for t in sorted(targets)]}')

['0', '4', '8', 'a', 'c', '10', '14', '16', '18', '1c', '1e', '20', '24', '28', '2a', '2c', '30', '34', '38', '3a', '3c', '40', '44', '48', '4c', '4e', '50', '54', '58', '5a', '5c', '60', '62', '64', '68', '6a', '6c', '70', '72', '74', '78', '7c', '7e', '80', '84', '86', '88', '8a', '8c', '90', '94', '98', '9c', 'a0', 'a4', 'a8', 'ac', 'b0', 'b4', 'b8', 'bc', 'c0', 'c4', 'c8', 'ca', 'cc', 'ce', 'd0', 'd4', 'd8', 'da', 'dc', 'e0', 'e8', 'fa', '122', '124', '126', '168', '17e', '196', '1a2', '1ba', '1c6', '1d2', '1d6', '1e4', '1e6', '1e8', '1ea', '1ec', '1f4', '1fc', '206', '218', '226', '240', '242', '27a', '282', '286', '28e', '292', '29a', '29c', '29e', '2a6', '2ba', '2c0', '2c2', '2c8', '2d0', '2dc', '2de', '2e4', '2ec', '2f8', '2fa', '300', '30c', '310', '318', '31a', '320', '322', '324', '32a', '332', '33a', '33e', '340', '34c', '356', '364', '36c', '378', '37a', '380', '38c', '390', '39c', '39e', '3a4', '3a6', '3a8', '3ae', '3b6', '3c2', '3c4', '3d0', '3da', '3e8', '3f0', '3fc', '

In [87]:
j=json.dumps(jsondata)

In [88]:
j[0:500]

'{"nodes": [{"name": "ln0", "group": 1}, {"name": "ln4", "group": 1}, {"name": "ln8", "group": 1}, {"name": "lna", "group": 1}, {"name": "lnc", "group": 1}, {"name": "ln10", "group": 1}, {"name": "ln14", "group": 1}, {"name": "ln16", "group": 1}, {"name": "ln18", "group": 1}, {"name": "ln1c", "group": 1}, {"name": "ln1e", "group": 1}, {"name": "ln20", "group": 1}, {"name": "ln24", "group": 1}, {"name": "ln28", "group": 1}, {"name": "ln2a", "group": 1}, {"name": "ln2c", "group": 1}, {"name": "ln30'

In [89]:
o=open('data.json','w')
o.write(j)
o.close()